<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/GROK4_FP_MARS_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_core -q
!pip install langchain -q
!pip install xai-sdk -q
!pip install langgraph -q

!pip install langchain_xai -q

!pip install -U langchain-community -q

In [6]:
import datetime
import random
import time
import json
import math
from datetime import timezone, timedelta
from typing import List, Dict, Any, Union, Optional

from pydantic import BaseModel, Field

from xai_sdk import Client
import os

from langchain_xai import ChatXAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor
from langchain_core.runnables import RunnablePassthrough
from langchain.agents.output_parsers.tools import ToolsAgentOutputParser
from langchain.agents.format_scratchpad.tools import format_to_tool_messages

# Configuration for XAI_API_KEY (adapt for your environment)
XAI_API_KEY = None
try:
    from google.colab import userdata
    XAI_API_KEY = userdata.get('XAI_KEY')
    if XAI_API_KEY:
        print("XAI_KEY found in Colab secrets.")
    else:
        print("WARNING: XAI_KEY not found in Colab secrets. Please ensure it is set.")
except ImportError:
    print("WARNING: Not running in Google Colab. Attempting to get XAI_KEY from environment variables.")
    XAI_API_KEY = os.environ.get('XAI_KEY')
    if XAI_API_KEY is None:
        print("WARNING: XAI_KEY not found in environment variables. Cannot run LLM-driven demonstration.")

# --- Define the new Mars-specific tools here ---

class MarsTrajectoryOptimizerToolInput(BaseModel):
    current_earth_position: Dict[str, Any] = Field(description="Current position of Earth (e.g., heliocentric coordinates).")
    mars_target_position: Dict[str, Any] = Field(description="Target position of Mars at desired arrival time.")
    launch_window_start: str = Field(description="Start date/time of the launch window (YYYY-MM-DD HH:MM:SS UTC).")
    launch_window_end: str = Field(description="End date/time of the launch window (YYYY-MM-DD HH:MM:SS UTC).")
    payload_mass_kg: float = Field(description="Mass of the spacecraft payload in kilograms.")
    propulsion_type: str = Field(description="Type of propulsion system (e.g., 'chemical', 'ion').")

@tool("MarsTrajectoryOptimizerTool_calculate_trajectory", args_schema=MarsTrajectoryOptimizerToolInput)
def calculate_trajectory(current_earth_position: Dict[str, Any], mars_target_position: Dict[str, Any], launch_window_start: str, launch_window_end: str, payload_mass_kg: float, propulsion_type: str) -> Dict[str, Any]:
    """Calculates an optimal interplanetary trajectory from Earth to Mars."""
    print(f" [Tool Call] Calculating trajectory from Earth ({current_earth_position.get('name', 'Earth')}) to Mars ({mars_target_position.get('name', 'Mars')})...")
    # --- Simulation Logic (simplified) ---
    estimated_transit_days = random.randint(180, 270) # 6 to 9 months
    delta_v_required_mps = random.uniform(4000, 6000) # m/s
    arrival_conditions = {"mars_orbit_insertion_burn_mps": random.uniform(800, 1200), "atmospheric_entry_velocity_mps": random.uniform(5500, 6000)}
    optimal_launch_date = "2025-09-01 00:00:00 UTC" # Example

    return {
        "optimal_launch_date_utc": optimal_launch_date,
        "estimated_transit_days": estimated_transit_days,
        "delta_v_required_mps": round(delta_v_required_mps, 2),
        "arrival_conditions": arrival_conditions,
        "notes": "Trajectory optimized for direct transfer. Further refinement with real-time solar wind data recommended."
    }

class SpaceWeatherToolInput(BaseModel):
    query_date_utc: str = Field(description="The date and time (YYYY-MM-DD HH:MM:SS UTC) for which to query space weather.")
    location: str = Field(description="Location in space to query (e.g., 'Earth_orbit', 'Mars_transfer_corridor').")

@tool("SpaceWeatherTool_get_space_weather", args_schema=SpaceWeatherToolInput)
def get_space_weather(query_date_utc: str, location: str) -> Dict[str, Any]:
    """Fetches space weather information (solar flares, radiation, meteoroids)."""
    print(f" [Tool Call] Fetching space weather for {location} on {query_date_utc}...")
    # --- Simulation Logic ---
    conditions = ["Low solar activity", "Moderate solar wind", "Minimal meteoroid risk"]
    if random.random() < 0.1: # 10% chance of a minor event
        conditions.append("Minor solar flare detected, increased radiation risk for 24 hours.")
    return {
        "date_utc": query_date_utc,
        "location": location,
        "conditions": conditions,
        "radiation_level_mSv_hr": random.uniform(0.05, 0.2), # milliSievert per hour
        "notes": "Monitor solar proton events for long-duration missions."
    }

class PropulsionSystemToolInput(BaseModel):
    aircraft_type: str = Field(description="Type of spacecraft (e.g., 'Starship', 'Orion').")
    delta_v_mps: float = Field(description="Required change in velocity in meters/second.")
    payload_mass_kg: float = Field(description="Mass of the spacecraft payload in kilograms.")
    engine_health_status: Optional[Dict[str, Any]] = Field(None, description="Current health status of engines.")

@tool("PropulsionSystemTool_calculate_fuel_burn", args_schema=PropulsionSystemToolInput)
def calculate_fuel_burn(aircraft_type: str, delta_v_mps: float, payload_mass_kg: float, engine_health_status: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    """Calculates fuel consumption for a given delta-v maneuver."""
    print(f" [Tool Call] Calculating fuel burn for {aircraft_type} for {delta_v_mps} m/s delta-v...")
    # --- Simulation Logic ---
    base_isp_seconds = 350 # Example Specific Impulse for chemical (higher for ion)
    if "ion" in aircraft_type.lower():
        base_isp_seconds = 3000 # Much higher for ion engines

    g0 = 9.80665 # standard gravity m/s^2
    initial_mass_kg = payload_mass_kg + random.uniform(5000, 10000) # Assume some dry mass

    # If engine health degraded, simulate reduced Isp or efficiency
    if engine_health_status and engine_health_status.get("degraded_engine"):
        base_isp_seconds *= 0.95 # 5% reduction

    final_mass_fraction = 1 / (math.exp(delta_v_mps / (base_isp_seconds * g0)))
    fuel_needed_kg = initial_mass_kg * (1 - final_mass_fraction)

    return {"fuel_needed_kg": round(fuel_needed_kg, 0), "initial_mass_kg": round(initial_mass_kg, 0), "isp_used": base_isp_seconds}

class LifeSupportToolInput(BaseModel):
    crew_count: int = Field(description="Number of crew members.")
    mission_duration_days: int = Field(description="Estimated mission duration in days.")
    current_resource_levels: Dict[str, float] = Field(description="Current levels of vital resources (e.g., 'oxygen_kg', 'water_liter', 'food_kg').")

@tool("LifeSupportTool_predict_consumption", args_schema=LifeSupportToolInput)
def predict_consumption(crew_count: int, mission_duration_days: int, current_resource_levels: Dict[str, float]) -> Dict[str, Any]:
    """Predicts life support resource consumption and alerts for shortages."""
    print(f" [Tool Call] Predicting life support consumption for {crew_count} crew over {mission_duration_days} days...")
    # --- Simulation Logic (simplified averages) ---
    oxygen_per_day_kg = 0.8 * crew_count
    water_per_day_liter = 3.0 * crew_count
    food_per_day_kg = 1.5 * crew_count

    predicted_consumption = {
        "oxygen_needed_kg": round(oxygen_per_day_kg * mission_duration_days, 2),
        "water_needed_liter": round(water_per_day_liter * mission_duration_days, 2),
        "food_needed_kg": round(food_per_day_kg * mission_duration_days, 2)
    }

    alerts = []
    if current_resource_levels.get("oxygen_kg", 0) < predicted_consumption["oxygen_needed_kg"] * 1.1: # 10% buffer
        alerts.append("WARNING: Oxygen levels projected to be critical by end of mission without resupply or recycling.")
    if current_resource_levels.get("water_liter", 0) < predicted_consumption["water_needed_liter"] * 1.1:
        alerts.append("WARNING: Water levels projected to be critical by end of mission.")

    return {"predicted_consumption": predicted_consumption, "alerts": alerts, "notes": "Resource calculations are estimates; actual usage may vary."}

# Collect all new tools
all_mars_tools = [
    calculate_trajectory,
    get_space_weather,
    calculate_fuel_burn,
    predict_consumption
]

# --- LLMAgent (remains largely the same, but binds to the new tools) ---
class LLMAgent:
    def __init__(self, api_key: str, verbose: bool = False):
        self.verbose = verbose
        if not api_key:
            raise ValueError("API key must be provided for LLMAgent.")
        self.llm = ChatXAI(model="grok-4", xai_api_key=api_key, temperature=0.5)

        self.llm_with_tools = self.llm.bind_tools(all_mars_tools)

        self.prompt = ChatPromptTemplate.from_messages(
            [
                ("system", "You are a highly intelligent space mission planning AI. Your goal is to assist in planning missions to Mars, considering all critical factors like trajectory, space weather, propulsion, and life support. You have access to specialized tools for these tasks. Provide clear, concise, and actionable mission briefings."),
                MessagesPlaceholder(variable_name="chat_history"),
                ("human", "{input}"),
                MessagesPlaceholder(variable_name="agent_scratchpad"),
            ]
        )

        self.agent_executor = AgentExecutor(
            agent=(
                RunnablePassthrough.assign(
                    agent_scratchpad=lambda x: format_to_tool_messages(x["intermediate_steps"])
                )
                | self.prompt
                | self.llm_with_tools
                | ToolsAgentOutputParser()
            ),
            tools=all_mars_tools,
            verbose=self.verbose,
            handle_parsing_errors=True
        )
        self._log("LLMAgent initialized with Langchain AgentExecutor for Mars planning.")

    def _log(self, message):
        if self.verbose:
            print(f" [LLMAgent] {message}")

    def run_inference(self, user_query: str, initial_context: dict) -> str:
        self._log("Starting inference run for new Mars mission query.")
        context_message = f"Initial planning context: {initial_context}"
        chat_history = [HumanMessage(content=context_message)]
        try:
            result = self.agent_executor.invoke({"input": user_query, "chat_history": chat_history})
            final_response_text = result.get("output", "No specific output from agent.")
            self._log("AgentExecutor finished.")
            return final_response_text
        except Exception as e:
            self._log(f"An error occurred during agent execution: {type(e).__name__} - {e}")
            if "Input to ChatPromptTemplate is missing variables" in str(e):
                return f"An internal configuration error occurred: {e}"
            return f"An internal error occurred during planning: {type(e).__name__}."


class FlightPlanningAgent:
    def __init__(self, api_key: str, name: str = "Mars Mission Planner (Grok-4)", verbose: bool = False):
        self.name = name
        self.verbose = verbose
        self.llm_agent = LLMAgent(api_key=api_key, verbose=verbose)
        self.history = []

    def _log(self, message):
        if self.verbose:
            print(f"[{self.name}] {message}")
        self.history.append(message)

    def plan_mars_mission(self, payload_kg: float, crew_count: int, target_launch_year: int, preferred_transit_duration_days: Optional[int] = None) -> str:
        # Define the EST timezone object (UTC-5)
        est_timezone = timezone(timedelta(hours=-5))
        # Updated to current time as per your context: Saturday, July 19, 2025, 3:46:30 PM EST
        # Ensure no leading zeros for month, day, hour, minute, second if they are single digits.
        current_datetime_est = datetime.datetime(2025, 7, 19, 15, 46, 30, tzinfo=est_timezone)

        user_request = f"Plan a mission to Mars for a {payload_kg} kg payload and {crew_count} crew members, targeting a launch around {target_launch_year}."
        if preferred_transit_duration_days:
            user_request += f" Preferred transit duration is around {preferred_transit_duration_days} days."
        self._log(f"Received user request: '{user_request}'")

        initial_context = {
            "current_time_est": current_datetime_est.strftime("%Y-%m-%d %H:%M:%S %Z%z"),
            "payload_mass_kg": payload_kg,
            "crew_count": crew_count,
            "target_launch_year": target_launch_year,
            "preferred_transit_duration_days": preferred_transit_duration_days,
            "earth_position_example": {"name": "Earth", "coordinates": "J2000_heliocentric_example"},
            "mars_position_example": {"name": "Mars", "coordinates": "J2000_heliocentric_example_target"}
        }

        final_briefing_response = self.llm_agent.run_inference(user_request, initial_context)
        self._log("Mars mission planning process driven by Grok-4 complete.")
        return final_briefing_response

# --- Demonstrate Usage (main block) ---
if __name__ == "__main__":
    print("\n" + "="*80 + "\n")
    print("--- Demonstrating Mars Mission Planner (Grok-4) ---")
    print("--- Powered by Grok-4 LLM (using Langchain's ChatXAI) ---")

    if not XAI_API_KEY:
        print("\nSkipping LLM-driven demonstration as XAI_API_KEY was not found.")
        print("Please ensure your 'XAI_KEY' is correctly set in Google Colab secrets or environment variables.")
    else:
        print("\n-- Scenario: Initial Mars Reconnaissance Mission --")
        mars_planner_agent = FlightPlanningAgent(api_key=XAI_API_KEY)

        # Example values for a reconnaissance mission
        payload = 1500.0 # kg
        crew = 0 # Uncrewed reconnaissance
        launch_year = 2033

        briefing = mars_planner_agent.plan_mars_mission(payload, crew, launch_year)

        print("\nFinal Briefing from Grok-powered Agent:")
        print(briefing)

        print("\n" + "="*80 + "\n")
        print("-- Scenario: Crewed Mars Transit Mission (with assumed initial resources) --")
        mars_planner_agent_crewed = FlightPlanningAgent(api_key=XAI_API_KEY, name="Crewed Mars Planner (Grok-4)")

        crew_payload = 25000.0 # kg (including habitat, life support, crew)
        num_crew = 4
        target_year = 2040

        briefing_crewed = mars_planner_agent_crewed.plan_mars_mission(crew_payload, num_crew, target_year)

        print("\nFinal Briefing from Grok-powered Agent (Crewed Mission):")
        print(briefing_crewed)

XAI_KEY found in Colab secrets.


--- Demonstrating Mars Mission Planner (Grok-4) ---
--- Powered by Grok-4 LLM (using Langchain's ChatXAI) ---

-- Scenario: Initial Mars Reconnaissance Mission --
 [Tool Call] Calculating trajectory from Earth (Earth) to Mars (Mars)...

Final Briefing from Grok-powered Agent:
### Mars Mission Briefing: Unmanned Payload Delivery (2033 Launch Target)

#### Mission Overview
- **Objective**: Deliver a 1500 kg unmanned payload (e.g., scientific instruments, rover components, or cargo) to Mars. No crew involved (crew count: 0), focusing on a direct transfer trajectory for efficiency.
- **Target Launch Year**: 2033 (aligned with optimal Earth-Mars alignment for minimal energy requirements).
- **Mission Type**: Uncrewed cargo/probe mission. Assumes chemical propulsion for reliability and cost-effectiveness.
- **Key Assumptions**: Based on provided context and tool optimizations. Positions used are heliocentric J2000 reference examples. Adjustments may be neede